<a href="https://colab.research.google.com/github/jithendra727/Bank-Account-System/blob/main/Sinhala_BiLSTM_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎙️ Sinhala-English Speech-to-Text — Custom BiLSTM Model
### Final Year Project | Train your own model from scratch on Kaggle data

**What this notebook builds:**
- ✅ MFCC feature extraction (NumPy only, no librosa) — **your code**
- ✅ Custom Sinhala-English vocabulary (English + full Sinhala script) — **your code**
- ✅ BiLSTM + CTC neural network — **your architecture**
- ✅ Training on 4 Kaggle datasets (Sinhala + English) — **your pipeline**
- ✅ Auto-saves to Google Drive (survives session restarts)
- ✅ Live microphone transcription after training

**Run order:** Cell 1 → 2 → 3 → 4 → 5 → 6 → 7 → 8 → 9 → 10 → 11 → 12 → 13

> ⚡ **Enable GPU first:** Runtime → Change runtime type → T4 GPU

In [1]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 1 — Install Dependencies
# Run ONCE per session. Takes ~2 minutes.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
!pip install torch torchaudio -q
!pip install numpy scipy matplotlib pandas tqdm -q
!pip uninstall -y kaggle 2>/dev/null
!pip install -U kaggle -q
!sudo apt-get install -y ffmpeg -q 2>/dev/null

import subprocess
v = subprocess.run(['ffmpeg','-version'], capture_output=True, text=True)
print("✅ ffmpeg:", v.stdout.split('\n')[0])

import torch
print("✅ PyTorch:", torch.__version__)
print("✅ CUDA available:", torch.cuda.is_available())
if not torch.cuda.is_available():
    print("⚠️  No GPU detected! Go to Runtime → Change runtime type → T4 GPU")
    print("   Training will be extremely slow on CPU.")
else:
    print("✅ GPU:", torch.cuda.get_device_name(0))
print("\n✅ All dependencies installed. Move to Cell 2.")

Found existing installation: kaggle 1.7.4.5
Uninstalling kaggle-1.7.4.5:
  Successfully uninstalled kaggle-1.7.4.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 9.4 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
✅ ffmpeg: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
✅ PyTorch: 2.10.0+cu128
✅ CUDA available: True
✅ GPU: Tesla T4

✅ All dependencies installed. Move to Cell 2.


In [2]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 2 — Import Libraries
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import os, json, time, subprocess, base64, shutil, random
from datetime import datetime
from scipy.io import wavfile
from tqdm import tqdm
from IPython.display import display, Javascript, Audio, HTML, clear_output
from google.colab import output, drive, files

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Libraries imported")
print(f"💻 Device: {DEVICE}")

✅ Libraries imported
💻 Device: cuda


In [4]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 3 — Mount Google Drive
# All model checkpoints + logs save here automatically.
# If session crashes, you restart from the last checkpoint.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
drive.mount('/content/drive')

# Project folder on your Drive
DRIVE_DIR = '/content/drive/MyDrive/SinhalaSTT'
os.makedirs(DRIVE_DIR, exist_ok=True)
os.makedirs(f'{DRIVE_DIR}/checkpoints', exist_ok=True)
os.makedirs(f'{DRIVE_DIR}/logs', exist_ok=True)

print(f"✅ Google Drive mounted")
print(f"📁 Project folder: {DRIVE_DIR}")
print(f"   Checkpoints → {DRIVE_DIR}/checkpoints/")
print(f"   Training logs → {DRIVE_DIR}/logs/")

# Check for existing checkpoints to resume training
existing = sorted([f for f in os.listdir(f'{DRIVE_DIR}/checkpoints') if f.endswith('.pth')])
if existing:
    print(f"\n♻️  Found {len(existing)} existing checkpoint(s):")
    for f in existing[-3:]:
        print(f"   - {f}")
    print("   → Training will resume from the latest checkpoint!")
else:
    print("\n📝 No checkpoints found — will train from scratch.")

Mounted at /content/drive
✅ Google Drive mounted
📁 Project folder: /content/drive/MyDrive/SinhalaSTT
   Checkpoints → /content/drive/MyDrive/SinhalaSTT/checkpoints/
   Training logs → /content/drive/MyDrive/SinhalaSTT/logs/

📝 No checkpoints found — will train from scratch.


In [5]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 4 — Kaggle API Setup
#
# IMPORTANT: Use YOUR NEW token (not the old exposed one).
# Get it from: https://www.kaggle.com/settings → API → Create New Token
#
# HOW TO ADD IT SAFELY (don't paste in code!):
#   1. Click the 🔑 key icon in the left sidebar (Colab Secrets)
#   2. Add secret name:  KAGGLE_USERNAME  → value: your_username
#   3. Add secret name:  KAGGLE_KEY       → value: your_new_token
#   4. Toggle "Notebook access" ON for both
#   Then uncomment the userdata lines below.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# ── Option A: Use Colab Secrets (RECOMMENDED — most secure) ──
from google.colab import userdata
try:
    os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    os.environ['KAGGLE_KEY']      = userdata.get('KAGGLE_KEY')
    print("✅ Kaggle credentials loaded from Colab Secrets")
except Exception:
    print("⚠️  Secrets not found. Using Option B below.")

# ── Option B: Direct (less secure — only use in private notebook) ──
# Uncomment and fill in ONLY if Option A didn't work:
# os.environ['KAGGLE_USERNAME'] = 'your_username_here'
# os.environ['KAGGLE_KEY']      = 'your_new_token_here'

# ── Verify connection ──
result = subprocess.run(['kaggle', 'datasets', 'list', '--max-size', '1'],
                        capture_output=True, text=True)
if 'Error' not in result.stderr and result.returncode == 0:
    print("✅ Kaggle API connected successfully!")
else:
    print("❌ Kaggle connection failed. Check your credentials.")
    print("   Error:", result.stderr[:200])

⚠️  Secrets not found. Using Option B below.
❌ Kaggle connection failed. Check your credentials.
   Error: 


In [6]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 5 — MFCC Feature Extractor (FROM SCRATCH)
# Pure NumPy. No librosa. Every step is your own math.
#
# Pipeline:
#   1. Pre-emphasis    : y[n] = x[n] - 0.97·x[n-1]
#   2. Framing         : 25ms windows, 10ms step
#   3. Hamming window  : 0.54 - 0.46·cos(2πn/(N-1))
#   4. FFT             : (1/N)|FFT(x)|²
#   5. Mel filterbank  : 40 triangular filters on Mel scale
#   6. Log compression : log(mel + ε)
#   7. DCT (MFCC)      : Σ log_mel·cos(π·k·(2n+1)/2M)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class MFCCExtractor:
    """MFCC feature extraction built entirely from scratch using NumPy."""

    def __init__(self, sr=16000, n_ceps=13, n_filters=40,
                 frame_len=0.025, frame_step=0.010, nfft=512, pre_emph=0.97):
        self.sr         = sr
        self.n_ceps     = n_ceps
        self.n_filters  = n_filters
        self.fl_samples = int(round(frame_len  * sr))
        self.fs_samples = int(round(frame_step * sr))
        self.nfft       = nfft
        self.pre_emph   = pre_emph
        self._fbank     = self._build_filterbank()   # pre-compute once

    # ── 1. Pre-emphasis ─────────────────────────────────────────────────────
    def _pre_emphasis(self, s):
        return np.concatenate([[s[0]], s[1:] - self.pre_emph * s[:-1]])

    # ── 2. Framing ──────────────────────────────────────────────────────────
    def _frame(self, s):
        n_frames = 1 + max(0, int(np.ceil((len(s) - self.fl_samples) / self.fs_samples)))
        pad      = (n_frames - 1) * self.fs_samples + self.fl_samples - len(s)
        s        = np.pad(s, (0, max(0, pad)))
        rows     = np.tile(np.arange(self.fl_samples), (n_frames, 1))
        cols     = np.tile(np.arange(0, n_frames * self.fs_samples, self.fs_samples),
                           (self.fl_samples, 1)).T
        return s[(rows + cols).astype(np.int32)]

    # ── 3. Hamming window ────────────────────────────────────────────────────
    def _hamming(self, N):
        n = np.arange(N)
        return 0.54 - 0.46 * np.cos(2 * np.pi * n / (N - 1))

    # ── 4. Power spectrum ────────────────────────────────────────────────────
    def _power(self, frames):
        return (1.0 / self.nfft) * np.abs(np.fft.rfft(frames, self.nfft)) ** 2

    # ── 5. Mel filterbank (pre-computed) ────────────────────────────────────
    @staticmethod
    def _hz2mel(f): return 2595.0 * np.log10(1.0 + f / 700.0)
    @staticmethod
    def _mel2hz(m): return 700.0 * (10.0 ** (m / 2595.0) - 1.0)

    def _build_filterbank(self):
        lo, hi = self._hz2mel(0), self._hz2mel(self.sr / 2)
        mels   = np.linspace(lo, hi, self.n_filters + 2)
        bins   = np.floor((self.nfft + 1) * self._mel2hz(mels) / self.sr).astype(int)
        fbank  = np.zeros((self.n_filters, self.nfft // 2 + 1))
        for m in range(self.n_filters):
            for k in range(bins[m],   bins[m+1]): fbank[m,k] = (k-bins[m])/(bins[m+1]-bins[m])
            for k in range(bins[m+1], bins[m+2]): fbank[m,k] = (bins[m+2]-k)/(bins[m+2]-bins[m+1])
        return fbank

    # ── 6+7. Log + DCT → MFCC ───────────────────────────────────────────────
    def _dct_mfcc(self, log_mel):
        M    = log_mel.shape[1]
        mfcc = np.zeros((log_mel.shape[0], self.n_ceps))
        for k in range(self.n_ceps):
            mfcc[:, k] = np.sum(
                log_mel * np.cos(np.pi * k * (2 * np.arange(M) + 1) / (2 * M)),
                axis=1)
        return mfcc

    def extract(self, signal, target_sr=None):
        """
        Full pipeline: raw audio (float32 numpy) → MFCC matrix (T × n_ceps).
        Handles any sample rate by resampling to self.sr first.
        """
        # Resample if needed
        if target_sr is not None and target_sr != self.sr:
            ratio  = self.sr / target_sr
            n_new  = int(len(signal) * ratio)
            signal = np.interp(np.linspace(0, len(signal)-1, n_new),
                               np.arange(len(signal)), signal)
        signal   = signal.astype(np.float32)
        emph     = self._pre_emphasis(signal)
        frames   = self._frame(emph)
        frames  *= self._hamming(self.fl_samples)
        power    = self._power(frames)
        mel      = np.dot(power, self._fbank.T)
        mel      = np.where(mel == 0, np.finfo(float).eps, mel)
        log_mel  = np.log(mel)
        return self._dct_mfcc(log_mel)

# ── Quick verification ───────────────────────────────────────────────────────
mfcc_ex = MFCCExtractor(sr=16000)
dummy   = np.random.randn(16000).astype(np.float32)
out     = mfcc_ex.extract(dummy)

print("━"*55)
print("  MFCC EXTRACTOR — VERIFIED ✅")
print("━"*55)
print(f"  Input  : 16,000 samples  (1 sec @ 16kHz)")
print(f"  Output : {out.shape[0]} frames × {out.shape[1]} coefficients")
print(f"  Filterbank: {mfcc_ex.n_filters} Mel triangular filters")
print(f"  Steps  : Pre-emphasis → Frame → Hamming → FFT")
print(f"           → Mel-filter → Log → DCT")
print("━"*55)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  MFCC EXTRACTOR — VERIFIED ✅
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Input  : 16,000 samples  (1 sec @ 16kHz)
  Output : 99 frames × 13 coefficients
  Filterbank: 40 Mel triangular filters
  Steps  : Pre-emphasis → Frame → Hamming → FFT
           → Mel-filter → Log → DCT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


In [7]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 6 — Bilingual Vocabulary
# Covers full Sinhala Unicode script + English + diacritics.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class BilingualVocab:
    """
    Complete Sinhala-English character vocabulary for CTC training.

    Includes:
    - English: a-z, digits 0-9, apostrophe
    - Sinhala: 18 vowels, 41 consonants, all diacritics (hal kirima,
               vowel marks, conjunct signs)
    - Special: space, <blank> (CTC), <unk>
    """

    def __init__(self):
        # ── English ─────────────────────────────────────────────────────────
        english = list("abcdefghijklmnopqrstuvwxyz0123456789'")

        # ── Sinhala vowels (independent) ────────────────────────────────────
        si_vowels = [
            'අ','ආ','ඇ','ඈ','ඉ','ඊ','උ','ඌ','ඍ','ඎ',
            'ඏ','ඐ','එ','ඒ','ඓ','ඔ','ඕ','ඖ'
        ]

        # ── Sinhala consonants ───────────────────────────────────────────────
        si_consonants = [
            'ක','ඛ','ග','ඝ','ඞ','ඟ','ච','ඡ','ජ','ඣ',
            'ඤ','ඥ','ඦ','ට','ඨ','ඩ','ඪ','ණ','ඬ','ත',
            'ථ','ද','ධ','න','ඳ','ප','ඵ','බ','භ','ම',
            'ඹ','ය','ර','ල','ව','ශ','ෂ','ස','හ','ළ','ෆ'
        ]

        # ── Sinhala diacritics (vowel marks + hal kirima + special signs) ───
        si_diacritics = [
            'ා','ැ','ෑ','ි','ී','ු','ූ','ෘ','ෙ','ේ',
            'ෛ','ො','ෝ','ෞ','ෟ','ං','ඃ','්',
            'ෲ','ෳ'
        ]

        # ── Special tokens ───────────────────────────────────────────────────
        special = [' ', '<blank>', '<unk>']

        # ── Build combined vocab ─────────────────────────────────────────────
        self.chars       = english + si_vowels + si_consonants + si_diacritics + special
        self.c2i         = {c: i for i, c in enumerate(self.chars)}
        self.i2c         = {i: c for i, c in enumerate(self.chars)}
        self.blank_idx   = self.c2i['<blank>']
        self.unk_idx     = self.c2i['<unk>']

    def __len__(self):
        return len(self.chars)

    def encode(self, text):
        """Text → list of integer indices. Unknown chars → <unk>."""
        result = []
        for ch in text.lower():
            result.append(self.c2i.get(ch, self.unk_idx))
        return result

    def decode(self, indices, remove_blanks=True):
        """Indices → text string."""
        skip = {self.blank_idx} if remove_blanks else set()
        return ''.join(self.i2c.get(i,'') for i in indices if i not in skip)

    def ctc_decode(self, indices):
        """
        Greedy CTC decode:
        1. Remove consecutive duplicate tokens
        2. Remove blank tokens
        """
        decoded, prev = [], None
        for idx in indices:
            if idx != prev and idx != self.blank_idx:
                decoded.append(idx)
            prev = idx
        return self.decode(decoded, remove_blanks=False)

# ── Build vocabulary ─────────────────────────────────────────────────────────
vocab = BilingualVocab()

print("━"*55)
print("  BILINGUAL VOCABULARY LOADED")
print("━"*55)
print(f"  Total characters : {len(vocab)}")
print(f"  English          : a-z, 0-9, apostrophe   = 37")
print(f"  Sinhala vowels   : independent form        = 18")
print(f"  Sinhala consonants                         = 41")
print(f"  Sinhala diacritics (marks, hal, anusvara)  = 20")
print(f"  Special (<blank>, <unk>, space)             = 3")
print(f"  Blank index (CTC): {vocab.blank_idx}")
print()
# Test encode/decode
test = "hello කොහොමද"
enc = vocab.encode(test)
dec = vocab.decode(enc)
print(f"  Encode test: '{test}'")
print(f"  → {enc[:8]}...")
print(f"  → Decoded back: '{dec}'")
print("━"*55)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  BILINGUAL VOCABULARY LOADED
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Total characters : 119
  English          : a-z, 0-9, apostrophe   = 37
  Sinhala vowels   : independent form        = 18
  Sinhala consonants                         = 41
  Sinhala diacritics (marks, hal, anusvara)  = 20
  Special (<blank>, <unk>, space)             = 3
  Blank index (CTC): 117

  Encode test: 'hello කොහොමද'
  → [7, 4, 11, 11, 14, 116, 55, 107]...
  → Decoded back: 'hello කොහොමද'
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


In [10]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 7 FIX — Updated for newer PyTorch
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class BiLSTM_CTC(nn.Module):
    """
    Bidirectional LSTM model with CTC loss for speech recognition.
    Designed for bilingual Sinhala-English character-level prediction.
    """
    def __init__(self, input_size=13, hidden_size=512, num_layers=4,
                 num_classes=None, dropout=0.3):
        super().__init__()
        assert num_classes is not None
        self.input_norm = nn.BatchNorm1d(input_size)
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.dropout    = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x, input_lengths=None):
        B, T, F = x.shape
        x_norm = self.input_norm(x.reshape(B*T, F)).reshape(B, T, F)

        if input_lengths is not None:
            packed = nn.utils.rnn.pack_padded_sequence(
                x_norm, input_lengths.cpu(), batch_first=True, enforce_sorted=False)
            lstm_out, _ = self.lstm(packed)
            lstm_out, _ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)
        else:
            lstm_out, _ = self.lstm(x_norm)

        out = self.layer_norm(lstm_out)
        out = self.dropout(out)
        logits = self.fc(out)
        log_probs = F.log_softmax(logits, dim=-1).permute(1, 0, 2)
        return log_probs

# ── Initialise model, loss, optimiser ─────────────────────────────
model = BiLSTM_CTC(
    input_size=13,
    hidden_size=512,
    num_layers=4,
    num_classes=len(vocab),
    dropout=0.3
).to(DEVICE)

criterion = nn.CTCLoss(blank=vocab.blank_idx, zero_infinity=True, reduction='mean')
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-5)

# Fixed: removed verbose=True (not supported in newer PyTorch)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)

n_params = sum(p.numel() for p in model.parameters())
print("━"*55)
print("  MODEL ARCHITECTURE")
print("━"*55)
print(model)
print()
print(f"  Total parameters: {n_params:,}")
print(f"  Vocab size:       {len(vocab)}")
print(f"  Device:           {DEVICE}")
print(f"  Optimiser:        AdamW  lr=3e-4  wd=1e-5")
print(f"  LR scheduler:     ReduceLROnPlateau (patience=3)")
print("━"*55)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  MODEL ARCHITECTURE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
BiLSTM_CTC(
  (input_norm): BatchNorm1d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(13, 512, num_layers=4, batch_first=True, dropout=0.3, bidirectional=True)
  (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=1024, out_features=119, bias=True)
)

  Total parameters: 21,181,585
  Vocab size:       119
  Device:           cuda
  Optimiser:        AdamW  lr=3e-4  wd=1e-5
  LR scheduler:     ReduceLROnPlateau (patience=3)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


In [11]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 8 — Download Kaggle Datasets
#
# Downloads all 4 datasets. Large ones (sinscribe 12GB)
# take 15-30 mins on Colab. Keep the tab open.
# Files stay on Colab disk until session ends.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

DATASETS = {
    'sinscribe': {
        'slug'   : 'seniruepasinghe/sinscribe-sinhala-stt',
        'dest'   : '/content/data/sinscribe',
        'size'   : '~12 GB',
        'lang'   : 'Sinhala',
        'samples': '161k'
    },
    'multispk': {
        'slug'   : 'keshan/multi-speaket-tts-dataset-sinhala',
        'dest'   : '/content/data/multispk',
        'size'   : '~1.7 GB',
        'lang'   : 'Sinhala',
        'samples': 'thousands'
    },
    'voicemakers': {
        'slug'   : 'safnask/sinhalatts-dataset-publication-by-voicemakers',
        'dest'   : '/content/data/voicemakers',
        'size'   : '~6 GB',
        'lang'   : 'Sinhala',
        'samples': '~4 speakers'
    },
    'ljspeech': {
        'slug'   : 'mathurinache/the-lj-speech-dataset',
        'dest'   : '/content/data/ljspeech',
        'size'   : '~3 GB',
        'lang'   : 'English',
        'samples': '13,100'
    }
}

# ── Choose which to download ─────────────────────────────────────────────────
# Comment out any you want to skip (e.g. skip sinscribe first to test faster)
TO_DOWNLOAD = ['multispk', 'ljspeech', 'voicemakers', 'sinscribe']  # sinscribe last (biggest)

print("━"*60)
print("  KAGGLE DATASET DOWNLOAD")
print("━"*60)

for key in TO_DOWNLOAD:
    ds = DATASETS[key]
    dest = ds['dest']

    # Skip if already downloaded (saves time on re-runs)
    if os.path.exists(dest) and len(os.listdir(dest)) > 0:
        print(f"✅ {key:15} already downloaded — skipping")
        continue

    os.makedirs(dest, exist_ok=True)
    print(f"\n📥 Downloading {key} ({ds['size']}, {ds['lang']}, {ds['samples']} samples)...")
    print(f"   This may take a while for large datasets. Keep the tab open!")

    t0 = time.time()
    result = subprocess.run(
        ['kaggle', 'datasets', 'download', '-d', ds['slug'],
         '-p', dest, '--unzip', '-q'],
        capture_output=True, text=True
    )
    elapsed = time.time() - t0

    if result.returncode == 0:
        # Count files downloaded
        n_files = sum(len(files) for _, _, files in os.walk(dest))
        total_mb = sum(
            os.path.getsize(os.path.join(r,f))
            for r,_,fs in os.walk(dest) for f in fs
        ) / 1e6
        print(f"✅ {key}: {n_files} files, {total_mb:.0f} MB in {elapsed:.0f}s")
    else:
        print(f"❌ {key} failed: {result.stderr[:200]}")

print("\n━"*30)
print("📂 Dataset directory overview:")
for key in TO_DOWNLOAD:
    dest = DATASETS[key]['dest']
    if os.path.exists(dest):
        print(f"\n  [{key}] {dest}")
        for item in sorted(os.listdir(dest))[:8]:
            full = os.path.join(dest, item)
            size = os.path.getsize(full)/1e6 if os.path.isfile(full) else 0
            tag = f'({size:.1f}MB)' if size > 0 else '(folder)'
            print(f"    {'📄' if os.path.isfile(full) else '📁'} {item} {tag}")

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  KAGGLE DATASET DOWNLOAD
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📥 Downloading multispk (~1.7 GB, Sinhala, thousands samples)...
   This may take a while for large datasets. Keep the tab open!
✅ multispk: 5367 files, 2392 MB in 43s

📥 Downloading ljspeech (~3 GB, English, 13,100 samples)...
   This may take a while for large datasets. Keep the tab open!
✅ ljspeech: 13102 files, 3801 MB in 77s

📥 Downloading voicemakers (~6 GB, Sinhala, ~4 speakers samples)...
   This may take a while for large datasets. Keep the tab open!
✅ voicemakers: 5580 files, 7352 MB in 150s

📥 Downloading sinscribe (~12 GB, Sinhala, 161k samples)...
   This may take a while for large datasets. Keep the tab open!
✅ sinscribe: 161297 files, 14506 MB in 297s

━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
📂 Dataset directory overview:

  [multispk] /content/data/multispk
    📄 file-mapping.json (1.2MB)
    📄 file_index.

In [17]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 9 FIX — Dataset Parser (handles all 4 datasets)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def find_audio_files(root, exts={'.wav','.mp3','.flac','.ogg'}):
    """Recursively find all audio files under root."""
    found = []
    for dirpath, _, filenames in os.walk(root):
        for f in filenames:
            if os.path.splitext(f)[1].lower() in exts:
                found.append(os.path.join(dirpath, f))
    return sorted(found)


# ── Dataset-specific loaders ────────────────────────────────────────

def load_ljspeech(root, max_samples=None):
    """LJ Speech: pipe-separated, no header, format: ID|transcript|normalized"""
    csv_path = os.path.join(root, 'LJSpeech-1.1', 'metadata.csv')
    audio_dir = os.path.join(root, 'LJSpeech-1.1', 'wavs')

    if not os.path.exists(csv_path):
        return []

    # Read with pipe separator and NO header
    df = pd.read_csv(csv_path, sep='|', header=None,
                     names=['id', 'transcript', 'normalized'])

    if max_samples:
        df = df.sample(min(len(df), max_samples), random_state=42)

    entries = []
    for _, row in df.iterrows():
        wav_path = os.path.join(audio_dir, f"{row['id']}.wav")
        if os.path.exists(wav_path):
            entries.append({
                'file': wav_path,
                'text': row['transcript'],
                'sr': 22050,  # LJSpeech is 22kHz
                'lang': 'en'
            })
    return entries


def load_sinscribe(root, max_samples=None):
    """sinscribe: CSV with audio in data/ subfolder"""
    csv_path = os.path.join(root, 'dataset.csv')
    audio_dir = os.path.join(root, 'data')

    if not os.path.exists(csv_path):
        return []

    df = pd.read_csv(csv_path, encoding='utf-8')

    if max_samples:
        df = df.sample(min(len(df), max_samples), random_state=42)

    # Common column names in sinscribe datasets
    possible_audio_cols = ['path', 'file', 'audio_path', 'filename', 'wav']
    possible_text_cols = ['text', 'transcript', 'sentence']

    audio_col = next((c for c in df.columns if c.lower() in possible_audio_cols), df.columns[0])
    text_col = next((c for c in df.columns if c.lower() in possible_text_cols), df.columns[-1])

    entries = []
    for _, row in df.iterrows():
        text = str(row.get(text_col, '')).strip()
        if not text or text == 'nan':
            continue

        audio_file = str(row.get(audio_col, ''))
        # Try multiple path combinations
        for attempt in [
            os.path.join(audio_dir, audio_file),
            os.path.join(audio_dir, os.path.basename(audio_file)),
            os.path.join(root, audio_file),
        ]:
            if os.path.exists(attempt):
                entries.append({
                    'file': attempt,
                    'text': text,
                    'sr': 16000,
                    'lang': 'si'
                })
                break

    return entries


def load_multispk(root, max_samples=None):
    """Multi-speaker TTS: TSV file with direct WAV references"""
    tsv_path = os.path.join(root, 'file_index.tsv')

    if not os.path.exists(tsv_path):
        return []

    df = pd.read_csv(tsv_path, sep='\t', encoding='utf-8')

    if max_samples:
        df = df.sample(min(len(df), max_samples), random_state=42)

    # Column detection
    audio_col = next((c for c in df.columns if 'file' in c.lower() or 'path' in c.lower()), df.columns[0])
    text_col = next((c for c in df.columns if 'text' in c.lower() or 'transcript' in c.lower()), df.columns[-1])

    entries = []
    for _, row in df.iterrows():
        text = str(row.get(text_col, '')).strip()
        if not text or text == 'nan':
            continue

        audio_file = str(row.get(audio_col, ''))
        wav_path = os.path.join(root, os.path.basename(audio_file))

        if os.path.exists(wav_path):
            entries.append({
                'file': wav_path,
                'text': text,
                'sr': 16000,
                'lang': 'si'
            })

    return entries


def load_voicemakers(root, max_samples=None):
    """VoiceMakers: Multiple speaker folders, each with metadata.csv"""
    entries = []

    # Find all speaker folders
    speaker_folders = [
        os.path.join(root, d) for d in os.listdir(root)
        if os.path.isdir(os.path.join(root, d))
    ]

    for speaker_folder in speaker_folders:
        # Each speaker has nested folder structure: Speaker/Speaker/metadata.csv
        nested = os.path.join(speaker_folder, os.path.basename(speaker_folder))
        csv_path = os.path.join(nested, 'metadata.csv')

        if not os.path.exists(csv_path):
            # Try direct metadata.csv
            csv_path = os.path.join(speaker_folder, 'metadata.csv')

        if not os.path.exists(csv_path):
            continue

        df = pd.read_csv(csv_path, sep='|', header=None,
                         names=['id', 'transcript', 'normalized'])

        # Audio files are in wavs/ subfolder
        audio_dir = os.path.join(os.path.dirname(csv_path), 'wavs')

        for _, row in df.iterrows():
            text = str(row['transcript']).strip()
            if not text or text == 'nan':
                continue

            wav_path = os.path.join(audio_dir, f"{row['id']}.wav")
            if os.path.exists(wav_path):
                entries.append({
                    'file': wav_path,
                    'text': text,
                    'sr': 44100,  # VoiceMakers is 44.1kHz
                    'lang': 'si'
                })

    if max_samples and len(entries) > max_samples:
        entries = random.sample(entries, max_samples)

    return entries


# ── Load all datasets ────────────────────────────────────────────────

print("━"*60)
print("  PARSING DATASETS")
print("━"*60)

all_entries = []

dataset_loaders = [
    ('sinscribe',   '/content/data/sinscribe',   load_sinscribe,   50000),
    ('ljspeech',    '/content/data/ljspeech',    load_ljspeech,    10000),
    ('multispk',    '/content/data/multispk',    load_multispk,    None),
    ('voicemakers', '/content/data/voicemakers', load_voicemakers, None),
]

for name, path, loader_func, max_s in dataset_loaders:
    if not os.path.exists(path) or not os.listdir(path):
        print(f"  ⚠️  {name}: not downloaded — skipping")
        continue

    print(f"\n  📂 Parsing [{name}]...")
    try:
        entries = loader_func(path, max_samples=max_s)
        all_entries.extend(entries)
        print(f"  ✅ {name}: {len(entries):,} entries loaded")
    except Exception as e:
        print(f"  ❌ {name} failed: {e}")

# Shuffle combined data
random.shuffle(all_entries)

print(f"\n━"*30)
print(f"  TOTAL ENTRIES: {len(all_entries):,}")
si_count = sum(1 for e in all_entries if e['lang']=='si')
en_count = sum(1 for e in all_entries if e['lang']=='en')
print(f"  Sinhala: {si_count:,} | English: {en_count:,}")

if len(all_entries) > 0:
    print(f"\n  📝 Sample entries:")
    for e in all_entries[:3]:
        print(f"    [{e['lang'].upper()}] {e['text'][:50]}...")
        print(f"           → {e['file']}")
else:
    print("\n  ❌ Still 0 entries. Run debug cell below:")
    print("     !ls -la /content/data/sinscribe/data/ | head -20")

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  PARSING DATASETS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  📂 Parsing [sinscribe]...
  ✅ sinscribe: 0 entries loaded

  📂 Parsing [ljspeech]...
  ✅ ljspeech: 10,000 entries loaded

  📂 Parsing [multispk]...
  ✅ multispk: 5,364 entries loaded

  📂 Parsing [voicemakers]...
  ✅ voicemakers: 5,000 entries loaded

━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
━
  TOTAL ENTRIES: 20,364
  Sinhala: 10,364 | English: 10,000

  📝 Sample entries:
    [SI] සෑම ළමයෙක් මේ වසරේ ඉදිරි ශ්‍රේණිගත චිත්‍ර ශිල්පියා...
           → /content/data/voicemakers/Dinithi-44100/Dinithi-44100/wavs/DIN_060_15.wav
    [SI] pn_sin_01_02177.wav...
           → /content/data/multispk/pn_sin_01_02177.wav
    [SI] මේ නිසා කෝට්ටේ රජතුමා උඩුගම්පළ සකලකලා වල්ලභ රජු මග...
           → /content/data/voicemakers/Harini-44100/Harini-44100/wavs/HN_119_09.wav


In [18]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 10 — PyTorch Dataset + DataLoader
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def load_audio_robust(path, target_sr=16000):
    """
    Load any audio file (wav/mp3/flac) as float32 numpy array.
    Uses ffmpeg for non-wav formats.
    Returns (audio_array, sample_rate) or (None, None) on failure.
    """
    try:
        ext = os.path.splitext(path)[1].lower()

        if ext == '.wav':
            sr, audio = wavfile.read(path)
            if audio.ndim > 1:          # stereo → mono
                audio = audio.mean(axis=1)
            audio = audio.astype(np.float32)
            if audio.max() > 1.0:
                audio /= 32768.0        # int16 → float32 normalise
        else:
            # Convert mp3/flac/ogg via ffmpeg
            tmp = '/tmp/_audio_load.wav'
            r = subprocess.run(
                ['ffmpeg', '-y', '-i', path, '-ar', str(target_sr),
                 '-ac', '1', '-f', 'wav', tmp],
                capture_output=True
            )
            if r.returncode != 0:
                return None, None
            sr, audio = wavfile.read(tmp)
            audio = audio.astype(np.float32) / 32768.0

        # Resample if needed
        if sr != target_sr:
            ratio = target_sr / sr
            n_new = int(len(audio) * ratio)
            audio = np.interp(
                np.linspace(0, len(audio)-1, n_new),
                np.arange(len(audio)), audio
            )
            sr = target_sr

        # Validate: must be at least 0.1s and at most 20s
        if len(audio) < 0.1 * sr or len(audio) > 20 * sr:
            return None, None

        return audio.astype(np.float32), sr

    except Exception:
        return None, None


class SpeechDataset(Dataset):
    """
    PyTorch Dataset that:
    1. Loads audio files from disk
    2. Runs MFCC extraction (from scratch)
    3. Encodes transcript text to integer labels
    4. Filters out bad files on the fly
    """

    def __init__(self, entries, vocab, mfcc_extractor, max_label_len=200):
        self.entries   = entries
        self.vocab     = vocab
        self.mfcc_ex   = mfcc_extractor
        self.max_label = max_label_len

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        entry = self.entries[idx]
        audio, sr = load_audio_robust(entry['file'])

        if audio is None:
            return None   # collate_fn will filter this

        mfcc  = self.mfcc_ex.extract(audio)                      # (T, 13)
        label = self.vocab.encode(entry['text'])[:self.max_label] # list of ints

        if len(mfcc) < 5 or len(label) < 1:
            return None

        return (
            torch.FloatTensor(mfcc),        # (T, 13)
            torch.LongTensor(label),         # (L,)
            mfcc.shape[0],                   # input_len
            len(label)                        # target_len
        )


def collate_fn(batch):
    """Pad variable-length sequences into fixed-size batch tensors."""
    # Filter out failed samples (None)
    batch = [b for b in batch if b is not None]
    if not batch:
        return None

    mfccs, labels, in_lens, tgt_lens = zip(*batch)

    # Pad MFCCs to max length in batch
    max_t   = max(m.shape[0] for m in mfccs)
    padded  = torch.zeros(len(batch), max_t, mfccs[0].shape[1])
    for i, m in enumerate(mfccs):
        padded[i, :m.shape[0]] = m

    # Concatenate all labels (CTC expects 1D target tensor)
    flat_labels = torch.cat([l for l in labels])

    return (
        padded,                              # (B, T_max, 13)
        flat_labels,                         # (sum of label lengths,)
        torch.IntTensor(in_lens),            # (B,)
        torch.IntTensor(tgt_lens)            # (B,)
    )


# ── Create train/validation split ────────────────────────────────────────────
random.shuffle(all_entries)
split       = int(0.95 * len(all_entries))
train_data  = all_entries[:split]
val_data    = all_entries[split:]

train_ds = SpeechDataset(train_data, vocab, mfcc_ex)
val_ds   = SpeechDataset(val_data,   vocab, mfcc_ex)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True,
                          collate_fn=collate_fn, num_workers=2,
                          pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=8, shuffle=False,
                          collate_fn=collate_fn, num_workers=2,
                          pin_memory=True)

print("━"*55)
print("  DATASET READY")
print("━"*55)
print(f"  Train samples  : {len(train_data):,}")
print(f"  Val samples    : {len(val_data):,}")
print(f"  Batch size     : 8")
print(f"  Train batches  : {len(train_loader):,}")
print(f"  Val batches    : {len(val_loader):,}")
print("━"*55)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  DATASET READY
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Train samples  : 19,345
  Val samples    : 1,019
  Batch size     : 8
  Train batches  : 2,419
  Val batches    : 128
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


In [19]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 11 — Evaluation Metric: Word Error Rate
# Implemented from scratch using dynamic programming.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def word_error_rate(ref, hyp):
    """
    Compute Word Error Rate (WER) using Wagner-Fischer algorithm.

    WER = (S + D + I) / N
    S = substitutions, D = deletions, I = insertions, N = ref words

    Implemented from scratch with dynamic programming — O(n·m) time.
    """
    r = ref.lower().strip().split()
    h = hyp.lower().strip().split()
    R, H = len(r), len(h)

    # DP table: dp[i][j] = min edits to align r[:i] with h[:j]
    dp = np.zeros((R+1, H+1), dtype=int)
    dp[0, :] = np.arange(H+1)
    dp[:, 0] = np.arange(R+1)

    for i in range(1, R+1):
        for j in range(1, H+1):
            if r[i-1] == h[j-1]:
                dp[i,j] = dp[i-1,j-1]
            else:
                dp[i,j] = 1 + min(dp[i-1,j], dp[i,j-1], dp[i-1,j-1])

    edits = dp[R, H]
    return (edits / max(R, 1)) * 100, edits, R

def batch_wer(refs, hyps):
    """Average WER across a list of reference/hypothesis pairs."""
    total_wer, total_edits, total_words = 0, 0, 0
    for r, h in zip(refs, hyps):
        w, e, n = word_error_rate(r, h)
        total_edits += e
        total_words += n
    return (total_edits / max(total_words, 1)) * 100

# ── Quick test ───────────────────────────────────────────────────────────────
tests = [
    ("hello how are you",   "hello how are you"),
    ("good morning everyone", "good morning everyoane"),
    ("the project is done",   "project done"),
]
print("━"*55)
print("  WER METRIC — VERIFIED")
print("━"*55)
for ref, hyp in tests:
    w, e, n = word_error_rate(ref, hyp)
    print(f"  REF: {ref}")
    print(f"  HYP: {hyp}")
    print(f"  WER: {w:.1f}%  ({e} edits / {n} words)")
    print()
print("━"*55)
print("  ✅ WER evaluator ready (DP from scratch)")

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  WER METRIC — VERIFIED
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  REF: hello how are you
  HYP: hello how are you
  WER: 0.0%  (0 edits / 4 words)

  REF: good morning everyone
  HYP: good morning everyoane
  WER: 33.3%  (1 edits / 3 words)

  REF: the project is done
  HYP: project done
  WER: 50.0%  (2 edits / 4 words)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  ✅ WER evaluator ready (DP from scratch)


In [21]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 7 FIX v2 — Variable name conflict resolved
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class BiLSTM_CTC(nn.Module):
    """
    Bidirectional LSTM model with CTC loss for speech recognition.
    Designed for bilingual Sinhala-English character-level prediction.
    """
    def __init__(self, input_size=13, hidden_size=512, num_layers=4,
                 num_classes=None, dropout=0.3):
        super().__init__()
        assert num_classes is not None
        self.input_norm = nn.BatchNorm1d(input_size)
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.dropout    = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x, input_lengths=None):
        # FIXED: renamed F to n_feats to avoid shadowing torch.nn.functional.F
        B, T, n_feats = x.shape
        x_norm = self.input_norm(x.reshape(B*T, n_feats)).reshape(B, T, n_feats)

        if input_lengths is not None:
            packed = nn.utils.rnn.pack_padded_sequence(
                x_norm, input_lengths.cpu(), batch_first=True, enforce_sorted=False)
            lstm_out, _ = self.lstm(packed)
            lstm_out, _ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)
        else:
            lstm_out, _ = self.lstm(x_norm)

        out = self.layer_norm(lstm_out)
        out = self.dropout(out)
        logits = self.fc(out)
        log_probs = F.log_softmax(logits, dim=-1).permute(1, 0, 2)
        return log_probs

# ── Re-initialize model with fix ────────────────────────────────
model = BiLSTM_CTC(
    input_size=13,
    hidden_size=512,
    num_layers=4,
    num_classes=len(vocab),
    dropout=0.3
).to(DEVICE)

criterion = nn.CTCLoss(blank=vocab.blank_idx, zero_infinity=True, reduction='mean')
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)

n_params = sum(p.numel() for p in model.parameters())
print("━"*55)
print("  MODEL RE-INITIALIZED (bug fixed)")
print("━"*55)
print(f"  Total parameters: {n_params:,}")
print(f"  Device: {DEVICE}")
print("━"*55)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  MODEL RE-INITIALIZED (bug fixed)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Total parameters: 21,181,585
  Device: cuda
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 12 — Training Loop
#
# Features:
#   ✅ Saves checkpoint to Google Drive every epoch
#   ✅ Resumes automatically from latest checkpoint
#   ✅ Plots training + validation loss live
#   ✅ Computes WER on validation set each epoch
#   ✅ Gradient clipping (prevents exploding gradients)
#   ✅ LR scheduler (reduces LR when loss plateaus)
#   ✅ Keeps best model separately as best_model.pth
#
# Expected training times on T4 GPU:
#   ~10k samples  : ~20 mins / epoch
#   ~50k samples  : ~2 hrs / epoch  (use fewer epochs)
#   ~160k samples : ~5 hrs / epoch  (use 3-5 epochs max)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

EPOCHS      = 15     # Reduce to 5 if dataset > 50k samples
GRAD_CLIP   = 5.0    # Max gradient norm (prevents exploding)
BEST_WER    = float('inf')
START_EPOCH = 0

CKPT_DIR    = f'{DRIVE_DIR}/checkpoints'
LOG_PATH    = f'{DRIVE_DIR}/logs/training_log.json'

# ── Resume from checkpoint if it exists ──────────────────────────────────────
existing_ckpts = sorted([f for f in os.listdir(CKPT_DIR) if f.startswith('epoch_')])
if existing_ckpts:
    latest = os.path.join(CKPT_DIR, existing_ckpts[-1])
    ckpt   = torch.load(latest, map_location=DEVICE)
    model.load_state_dict(ckpt['model'])
    optimizer.load_state_dict(ckpt['optimizer'])
    START_EPOCH = ckpt['epoch'] + 1
    BEST_WER    = ckpt.get('best_wer', float('inf'))
    print(f"♻️  Resumed from checkpoint: {existing_ckpts[-1]}")
    print(f"   Starting at epoch {START_EPOCH+1}, best WER so far: {BEST_WER:.1f}%")
else:
    print("📝 Starting training from scratch")

# ── Logging setup ─────────────────────────────────────────────────────────────
train_losses, val_losses, val_wers = [], [], []
if os.path.exists(LOG_PATH):
    with open(LOG_PATH) as f:
        log = json.load(f)
        train_losses = log.get('train_loss', [])
        val_losses   = log.get('val_loss',   [])
        val_wers     = log.get('val_wer',    [])


def run_epoch(loader, training=True):
    """Run one full pass through the dataset."""
    model.train() if training else model.eval()
    total_loss, n_batches = 0.0, 0

    context = torch.enable_grad() if training else torch.no_grad()
    with context:
        for batch in tqdm(loader, desc='Train' if training else 'Val ',
                          leave=False, ncols=80):
            if batch is None:
                continue
            mfccs, labels, in_lens, tgt_lens = batch
            mfccs  = mfccs.to(DEVICE)
            labels = labels.to(DEVICE)

            log_probs = model(mfccs, in_lens)   # (T, B, C)

            # CTC loss
            loss = criterion(log_probs, labels, in_lens, tgt_lens)

            if torch.isnan(loss) or torch.isinf(loss):
                continue

            if training:
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
                optimizer.step()

            total_loss += loss.item()
            n_batches  += 1

    return total_loss / max(n_batches, 1)


def evaluate_wer(loader, n_samples=50):
    """Decode predictions and compute WER on a sample of the val set."""
    model.eval()
    refs, hyps = [], []
    count = 0

    with torch.no_grad():
        for batch in loader:
            if batch is None or count >= n_samples:
                break
            mfccs, labels, in_lens, tgt_lens = batch
            mfccs = mfccs.to(DEVICE)

            log_probs = model(mfccs)                    # (T, B, C)
            preds     = log_probs.argmax(dim=2)         # (T, B)
            preds     = preds.permute(1, 0).cpu().numpy()  # (B, T)

            # Reconstruct reference labels from flattened tensor
            label_list = []
            offset = 0
            for tl in tgt_lens:
                label_list.append(labels[offset:offset+tl].tolist())
                offset += tl

            for i in range(len(label_list)):
                ref_text = vocab.decode(label_list[i])
                hyp_text = vocab.ctc_decode(preds[i].tolist())
                refs.append(ref_text)
                hyps.append(hyp_text)
                count += 1

    return batch_wer(refs, hyps), refs[:3], hyps[:3]


# ── Main training loop ────────────────────────────────────────────────────────
print("━"*60)
print(f"  TRAINING — {EPOCHS} epochs  |  {len(train_loader)} batches/epoch")
print(f"  Dataset: {len(train_data):,} train + {len(val_data):,} val samples")
print(f"  Checkpoints → {CKPT_DIR}")
print("━"*60)

for epoch in range(START_EPOCH, EPOCHS):
    t_start = time.time()

    # ── Train ──
    train_loss = run_epoch(train_loader, training=True)

    # ── Validate ──
    val_loss = run_epoch(val_loader, training=False)
    wer, sample_refs, sample_hyps = evaluate_wer(val_loader)

    # ── LR step ──
    scheduler.step(val_loss)

    # ── Record ──
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_wers.append(wer)

    elapsed = time.time() - t_start
    lr_now  = optimizer.param_groups[0]['lr']

    print(f"\nEpoch {epoch+1:02d}/{EPOCHS}  [{elapsed:.0f}s]")
    print(f"  Train Loss: {train_loss:.4f}  |  Val Loss: {val_loss:.4f}  |  WER: {wer:.1f}%  |  LR: {lr_now:.2e}")
    print(f"  Sample — REF: {sample_refs[0][:60] if sample_refs else '—'}")
    print(f"         — HYP: {sample_hyps[0][:60] if sample_hyps else '—'}")

    # ── Save checkpoint to Drive ──
    ckpt_path = f'{CKPT_DIR}/epoch_{epoch+1:03d}_wer{wer:.1f}.pth'
    torch.save({
        'epoch'    : epoch,
        'model'    : model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'train_loss': train_loss,
        'val_loss'  : val_loss,
        'wer'       : wer,
        'best_wer'  : BEST_WER,
        'vocab_size': len(vocab),
    }, ckpt_path)
    print(f"  💾 Checkpoint saved → Drive")

    # ── Keep best model ──
    if wer < BEST_WER:
        BEST_WER = wer
        shutil.copy(ckpt_path, f'{DRIVE_DIR}/best_model.pth')
        print(f"  🏆 New best model! WER: {wer:.1f}%")

    # ── Save log ──
    with open(LOG_PATH, 'w') as f:
        json.dump({'train_loss': train_losses, 'val_loss': val_losses,
                   'val_wer': val_wers}, f)

    # ── Live loss plot ──
    if len(train_losses) >= 2:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))
        fig.patch.set_facecolor('#0f172a')
        for ax in (ax1, ax2):
            ax.set_facecolor('#1e293b')
            ax.tick_params(colors='#94a3b8')
            ax.spines[:].set_color('#334155')

        ax1.plot(train_losses, label='Train', color='#3b82f6', lw=2)
        ax1.plot(val_losses,   label='Val',   color='#f59e0b', lw=2)
        ax1.set_title('CTC Loss', color='white', fontweight='bold')
        ax1.set_xlabel('Epoch', color='#94a3b8')
        ax1.legend()
        ax1.grid(alpha=0.2)

        ax2.plot(val_wers, color='#10b981', lw=2, marker='o', markersize=4)
        ax2.set_title('Validation WER (%)', color='white', fontweight='bold')
        ax2.set_xlabel('Epoch', color='#94a3b8')
        ax2.set_ylabel('WER %', color='#94a3b8')
        ax2.axhline(y=BEST_WER, color='#ef4444', ls='--', lw=1, label=f'Best: {BEST_WER:.1f}%')
        ax2.legend(); ax2.grid(alpha=0.2)

        plt.tight_layout()
        plt.show()
        clear_output(wait=True)

print(f"\n{'━'*60}")
print(f"  TRAINING COMPLETE")
print(f"  Best WER: {BEST_WER:.1f}%")
print(f"  Best model: {DRIVE_DIR}/best_model.pth")
print(f"{'━'*60}")

📝 Starting training from scratch
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  TRAINING — 15 epochs  |  2419 batches/epoch
  Dataset: 19,345 train + 1,019 val samples
  Checkpoints → /content/drive/MyDrive/SinhalaSTT/checkpoints
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Train:   0%|                                           | 0/2419 [00:00<?, ?it/s]/tmp/ipython-input-1737617612.py:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, audio = wavfile.read(path)
/tmp/ipython-input-1737617612.py:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, audio = wavfile.read(path)
Val :   0%|                                             | 0/128 [00:00<?, ?it/s]/tmp/ipython-input-1737617612.py:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, audio = wavfile.read(path)
/tmp/ipython-input-1737617612.py:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, audio = wavfile.read(path)
/tmp/ipython-input-1737617612.py:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, audio = wavfile.read(path)
/tmp/ipython-input-1737617612.py:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, audio = wavfile.read(path)



Epoch 01/15  [1669s]
  Train Loss: 2.9762  |  Val Loss: 1.2856  |  WER: 91.8%  |  LR: 3.00e-04
  Sample — REF: sin<unk>4191<unk>5072534842<unk>wav
         — HYP: sin<unk>6<unk>wav
  💾 Checkpoint saved → Drive
  🏆 New best model! WER: 91.8%


Train:   0%|                                           | 0/2419 [00:00<?, ?it/s]/tmp/ipython-input-1737617612.py:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, audio = wavfile.read(path)
/tmp/ipython-input-1737617612.py:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, audio = wavfile.read(path)
Train:  85%|███████████████████████████▏    | 2054/2419 [22:57<03:25,  1.78it/s]

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 13 — Live Microphone Transcription
#
# Run AFTER training. Loads the best model and transcribes
# your microphone input continuously until you click STOP.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# ── Load best trained model ───────────────────────────────────────────────────
best_path = f'{DRIVE_DIR}/best_model.pth'
if os.path.exists(best_path):
    ckpt = torch.load(best_path, map_location=DEVICE)
    model.load_state_dict(ckpt['model'])
    print(f"✅ Loaded best model (WER: {ckpt.get('wer', '?'):.1f}%)")
else:
    print("⚠️  No best_model.pth found. Use model from training (Cell 12).")

model.eval()

# ── Audio capture bridge ──────────────────────────────────────────────────────
def record_chunk(seconds=5, sr=16000):
    js = Javascript(f"""
    async function rec() {{
        const stream = await navigator.mediaDevices.getUserMedia({{
            audio: {{ sampleRate: 16000, channelCount: 1,
                      echoCancellation: true, noiseSuppression: true }}
        }});
        const mr = new MediaRecorder(stream, {{ mimeType: 'audio/webm;codecs=opus' }});
        let chunks = [];
        mr.ondataavailable = e => chunks.push(e.data);
        mr.start();
        await new Promise(r => setTimeout(r, {seconds * 1000}));
        mr.stop();
        return new Promise(r => {{
            mr.onstop = async () => {{
                const b = new Blob(chunks, {{ type: 'audio/webm' }});
                const rd = new FileReader();
                rd.readAsDataURL(b);
                rd.onloadend = () => {{ stream.getTracks().forEach(t => t.stop()); r(rd.result); }};
            }};
        }});
    }}
    """)
    display(js)
    data = output.eval_js('rec()')
    if not data: return None, None
    raw  = base64.b64decode(data.split(',')[1])
    with open('/tmp/live.webm','wb') as f: f.write(raw)
    subprocess.run(['ffmpeg','-y','-i','/tmp/live.webm','-ar',str(sr),
                    '-ac','1','/tmp/live.wav'], capture_output=True)
    sr2, audio = wavfile.read('/tmp/live.wav')
    return audio.astype(np.float32) / 32768.0, sr2

def transcribe_audio(audio, sr):
    """Run the full pipeline: audio → MFCC → model → text."""
    # Voice Activity Detection
    rms = np.sqrt(np.mean(audio**2))
    if rms < 0.015:
        return None, rms   # silence

    mfcc     = mfcc_ex.extract(audio)
    feat     = torch.FloatTensor(mfcc).unsqueeze(0).to(DEVICE)  # (1, T, 13)

    with torch.no_grad():
        log_probs = model(feat)                          # (T, 1, C)

    preds = log_probs.argmax(dim=2).squeeze(1).cpu().numpy()
    text  = vocab.ctc_decode(preds.tolist())
    return text, rms

# ── Stop button ───────────────────────────────────────────────────────────────
stop = {'flag': False}
def do_stop(): stop['flag'] = True
output.register_callback('stop_live', do_stop)

# ── UI ────────────────────────────────────────────────────────────────────────
display(HTML("""
<div style="font-family:'Segoe UI',sans-serif;background:#0f172a;color:#e2e8f0;
  padding:24px;border-radius:14px;border:1px solid #334155;margin:10px 0;">
  <div style="display:flex;align-items:center;gap:14px;margin-bottom:18px;">
    <div style="width:12px;height:12px;border-radius:50%;background:#ef4444;
      animation:pulse 1.2s infinite;"></div>
    <span style="font-size:19px;font-weight:700;">🎙️ LIVE Sinhala-English Transcription</span>
    <button onclick="google.colab.kernel.invokeFunction('stop_live',[],{})"
      style="margin-left:auto;padding:9px 20px;background:#ef4444;color:white;
        border:none;border-radius:8px;cursor:pointer;font-weight:700;font-size:14px;">
      ⏹ STOP</button>
  </div>
  <div id="st" style="color:#64748b;font-size:12px;margin-bottom:14px;font-family:monospace;">
    ⏳ Starting...</div>
  <div id="lt" style="font-size:22px;font-weight:500;line-height:1.5;color:#f1f5f9;
    background:#1e293b;padding:16px;border-radius:10px;border-left:4px solid #3b82f6;
    min-height:60px;margin-bottom:14px;">—</div>
  <div id="ht" style="font-size:13px;color:#94a3b8;max-height:200px;overflow-y:auto;
    line-height:2;background:#0f172a;padding:10px 14px;border-radius:8px;"></div>
</div>
<style>@keyframes pulse{0%,100%{opacity:1}50%{opacity:0.3}}</style>
"""))

def upd(live, hist_html, status):
    display(Javascript(f"""
    (function(){{
        var a=document.getElementById('lt'),b=document.getElementById('ht'),
            c=document.getElementById('st');
        if(a) a.innerHTML={json.dumps(live or '—')};
        if(b) b.innerHTML={json.dumps(hist_html)};
        if(c) c.innerHTML={json.dumps(status)};
    }})();
    """))

# ── Session loop ──────────────────────────────────────────────────────────────
history, chunk_n, elapsed = [], 0, 0
full_transcript = []

print("✅ Starting — speak into your microphone!")
print("   Click STOP to end the session.\n")

while not stop['flag']:
    chunk_n += 1
    elapsed += 5
    mm, ss = elapsed // 60, elapsed % 60
    upd("🎤 Listening...", '<br>'.join(history[-12:]),
        f"🔴 REC  Chunk #{chunk_n}  |  {mm:02d}:{ss:02d}")

    audio, sr = record_chunk(seconds=5)
    if audio is None: continue

    upd("⏳ Transcribing...", '<br>'.join(history[-12:]),
        f"🔄 Processing chunk #{chunk_n}...")

    text, rms = transcribe_audio(audio, sr)

    if text is None:
        upd(history[-1].split('  ')[-1] if history else '—',
            '<br>'.join(history[-12:]),
            f"🤫 Silence (energy={rms:.3f})  |  Chunk #{chunk_n}")
        continue

    ts  = datetime.now().strftime('%H:%M:%S')
    row = f"<span style='color:#64748b'>[{ts}]</span>  <span style='color:#e2e8f0'>{text}</span>"
    history.append(row)
    full_transcript.append(f"[{ts}]  {text}")

    upd(text, '<br>'.join(history[-12:]),
        f"✅ Done  |  Chunk #{chunk_n}  |  {mm:02d}:{ss:02d}")

    if stop['flag']: break

# ── Save transcript ───────────────────────────────────────────────────────────
print("\n" + "━"*60)
print("  SESSION COMPLETE")
print("━"*60)
print('\n'.join(full_transcript))

if full_transcript:
    ts_file = f"{DRIVE_DIR}/transcript_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    with open(ts_file, 'w', encoding='utf-8') as f:
        f.write("\n".join(full_transcript))
    print(f"\n💾 Transcript saved → {ts_file}")

stop['flag'] = False   # reset for next run